In [1]:
# config


pdagent_type = 'auto' #one of: auto, manual
llm_type = 'gpt-3.5-turbo' #one of: gpt-3.5-turbo-0613, gpt-3.5-turbo-0125, gpt-3.5-turbo, gpt-4o
initialization_params = {} #eg json version

### Imports & data setup

In [2]:
import pandas as pd
import os
import json
import openai
import chromadb
import gradio as gr

from langchain.chains.query_constructor.base import AttributeInfo
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.tools import PythonAstREPLTool
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

if not os.environ.get("OPENAI_API_KEY"):
    import key
    key.init()
    assert os.environ.get('OPENAI_API_KEY')

/Users/snc/opt/anaconda3/envs/giss/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = './model_experiment_fields_ScenarioMIP_CMIP_filename_dates.csv'
df = pd.read_csv(path)
df['collection'] = 'giss_cmip6'
df = df[['collection', 'MIP', 'model', 'experiment', 'variant', 'tableID', 'variable', 'grid', 'version', 'start_YM', 'end_YM', 'filename']]
df.columns = ['collection', 'MIP', 'model', 'experiment', 'variant', 'temporal resolution', 'variable', 'grid', 'version', 'start year', 'end year', 'filename']
df = df.astype(str)

def url(x):
    cols = '/'.join(x)
    return 'portal.nccs.nasa.gov/datashare/' + cols
df['URL'] = df.apply(lambda x: url(x), axis=1)

df['start year'] = df['start year'].apply(lambda x: x[:4]) #+'-'+x[4:])
df['end year'] = df['end year'].apply(lambda x: x[:4]) #+'-'+x[4:])

oldn = df.shape[0]
df = df.sort_values(df.columns.to_list(), ascending=True).drop_duplicates(
    subset=set(df.columns.to_list())-set(['version', 'filename', 'URL']),
    ignore_index=True, keep='last')
print('removed', oldn-df.shape[0], 'rows corresponding to old-version datasets')

path = '../../cmip6-cmor-tables/Tables'
jdfs = []
for fname in os.listdir(path):
    with open(os.path.join(path, fname)) as j:
        d = json.load(j)
    
    try: 
        jdf = pd.DataFrame.from_dict(d['variable_entry'], orient='index')
    except:
        print('skipping', fname, 'due to formatting issue')
        continue    
    
    jdf['temporal resolution'] = fname[len('CMIP6_'):-len('.json')]
    jdfs.append(jdf)
    
varsdf = pd.concat(jdfs)
varsdf.rename(columns={'out_name': 'variable'}, inplace=True)
varsdf = varsdf.drop_duplicates(subset=['variable', 'temporal resolution'])

dfm = df.merge(varsdf, how='left')
dfm.drop_duplicates(subset=set(dfm.columns.to_list())-set(['dimensions'])).shape
dfmsub = dfm.drop(['temporal resolution', 'grid', 'version', 'type', 'positive', 'valid_min', 'valid_max', 
        'ok_min_mean_abs', 'ok_max_mean_abs', 'flag_values', 'flag_meanings', 'collection', 'variant', 
        'filename', 'modeling_realm', 'standard_name', 'cell_methods', 'cell_measures', 'dimensions'], axis=1)

removed 6985 rows corresponding to old-version datasets
skipping CMIP6_coordinate.json due to formatting issue
skipping CMIP6_input_example.json due to formatting issue
skipping CMIP6_CV.json due to formatting issue
skipping CMIP6_formula_terms.json due to formatting issue


### LangChain setup

In [4]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "summarize the user's request in a few key words without punctuation. "
    "Do NOT attempt to answer the user's request. "
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
model = ChatOpenAI(model=llm_type, temperature=0)
chain_standalone = contextualize_q_prompt | model | StrOutputParser()

In [5]:
if pdagent_type == 'auto':
    # class create_pd_agent_that_returns_just_dataframe:
    #     def __init__(self, model, df):
    #         system = f"""
    #             You are a climate scientist with a pandas dataframe `df` that lists and describes all datasets 
    #             within CMIP6. Here is the output of `df.head().to_markdown:` \
    #             {df.head().to_markdown()} \
    #             Given a colleague's query, EXECUTE Python code to find ROWs of `df` corresponding 
    #             to CMIP6 datasets *similar* to the query (not necessarily an exact match!). \
    #                 Return ONLY the entire rows of `df` and nothing else. EXECUTE the code yourself!!
    #                 Do it yourself!!"""
    #         self.pdagent = create_pandas_dataframe_agent(
    #             model,
    #             df,
    #             prefix=system,
    #             verbose=True,
    #             agent_type=AgentType.OPENAI_FUNCTIONS,
    #             allow_dangerous_code=True,
    #             handle_parsing_errors="You you believe that the data doesn't contain the information you are looking for, please try rephrasing your question.",
    #             return_intermediate_steps=True,
    #             response_format={ "type": "json_object" },
    #         )
    #     def invoke(self, query):
    #         # return self.pdagent.invoke(query)
    #         try:
    #             result = self.pdagent.invoke(query)
    #             return result['intermediate_steps'][0][1]
    #         except:
    #             return pd.DataFrame([])
            
    
    system = f"""
        Return ONLY the entire relevant rows of `df` and nothing else. EXECUTE the code yourself!!
            Do it yourself!!"""
    model = ChatOpenAI(temperature=0, model=llm_type)
    pdagent = create_pandas_dataframe_agent(
                model,
                dfmsub,
                prefix=system,
                verbose=True,
                agent_type=AgentType.OPENAI_FUNCTIONS,
                allow_dangerous_code=True,
                handle_parsing_errors="You you believe that the data doesn't contain the information you are looking for, please try rephrasing your question.",
                return_intermediate_steps=True,
                response_format={ "type": "json_object" },
            )
    
elif pdagent_type == 'manual':
    llm=ChatOpenAI(temperature=0, model="gpt-4o", )
    tool = PythonAstREPLTool(locals={"df": dfmsub})
    llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
    parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

    system = f"""
    You are a climate scientist with a pandas dataframe `df` that lists and describes all datasets 
    within CMIP6. Here is the output of `df.head().to_markdown:` \
    {dfmsub.head().to_markdown()} \
    Given a colleague's query, find ROWs of `df` corresponding 
    to CMIP6 datasets *similar* to the query (not necessarily an exact match!). \
        Return ONLY the entire rows of `df` and nothing else. EXECUTE the code yourself!!
    """
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
    pdagent = prompt | llm_with_tools | parser | tool

/Users/snc/opt/anaconda3/envs/giss/lib/python3.11/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:282: UserWarning: Received additional kwargs {'handle_parsing_errors': "You you believe that the data doesn't contain the information you are looking for, please try rephrasing your question.", 'response_format': {'type': 'json_object'}} which are no longer supported.
  warnings.warn(


### Evaluate

In [6]:
queries = pd.read_excel('evals.xlsx')
queries.head()

,annotator,src,query,variable,frequency,start year,end year,MIP,experiment
0,Sonia,Us,What are some datasets relevant to extreme wea...,"pr, ua, etc",NaN,NaN,NaN,NaN,NaN
1,Sonia,Us,"Plot the 10, 20, 50, and 100 yr return period ...",pr,day,NaN,NaN,historical,NaN
2,Sonia,Us,Show me all the times where Ames Iowa has more...,tas,day,1980.0,2014.0,CMIP,historical
3,Sonia,Us,Show me the average annual number of days per ...,tasmax,day,2050.0,2070.0,ScenarioMIP,ssp<NNN>
4,Sonia,Us,You just showed me average annual number of da...,tasmax,day,2050.0,2070.0,ScenarioMIP,ssp<NNN>


In [7]:
def to_dictsets(r):
    for k,v in r.to_dict().items(): 
        if type(v)==str and ',' in v and k != 'query':
            r[k] = set(v.split(', '))
    return r

In [8]:
# %%capture
outs = []
for i, r in queries.iterrows():
    r = to_dictsets(r)
    conversation = {'input': r['query'], 'chat_history': []}
    standalone = chain_standalone.invoke(conversation)
    print(standalone)
    # model = ChatOpenAI(temperature=0, model=llm_type)
    # pdagent = create_pd_agent_that_returns_just_dataframe(model, dfmsub)
    # out = pdagent.invoke(standalone+'?')
    response = pdagent.invoke(standalone+'?')
    if pdagent_type == 'auto':
        try:
            outs.append(response)
        except:
            print('failed to provide dataframe')
            outs.append(response)
    else:
        outs.append(response)

datasets relevant extreme weather


> Entering new AgentExecutor chain...
To find datasets relevant to extreme weather, you can search for climate datasets that include variables related to extreme weather events such as heatwaves, heavy rainfall, droughts, hurricanes, and storms. One common source for climate datasets is the CMIP6 (Coupled Model Intercomparison Project Phase 6) database, which provides a wide range of climate model simulations.

You can explore the CMIP6 database or other climate data repositories to find specific variables related to extreme weather events. Additionally, you can look for datasets from organizations like NOAA (National Oceanic and Atmospheric Administration) or NASA (National Aeronautics and Space Administration) that focus on climate and weather research.

If you have access to a specific climate data platform or database, you can search for keywords like "extreme weather," "climate extremes," "heatwaves," "heavy precipitation," "tropical cyclones," 

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


There are no rows in the DataFrame where the variable is "Surface Air Temperature" and the units are in Kelvin.

> Finished chain.
average annual days rainfall exceeds 1mm globally 1980-2014 uncoupled run


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['experiment']=='uncoupled') & (df['variable']=='pr') & (df['start year']>=1980) & (df['end year']<=2014)]['frequency'].value_counts()"}`


Series([], Name: count, dtype: int64)There are no records in the dataset for the average annual days rainfall exceeding 1mm globally from 1980 to 2014 in uncoupled runs.

> Finished chain.
Future suitable wheat growing places


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['variable'].unique()"}`
responded: I need to analyze the data to identify future suitable wheat growing places. Let me first check the available variables in the dataset.

['bldep' 'cltc' 'cod' 'ptp' 'tatp' 'ttop' 'ccb' 'cct' 'ci' 'cl' 'cli'
 'cliv

In [9]:
non_score_cols = set(['annotator', 'src', 'query'])
pts= 0
valids = 0
for i in range(len(outs)):
    out = outs[i]
    r = queries.iloc[i]
    if type(out) != pd.core.frame.DataFrame or len(out) == 0:
        continue
    
    valids += 1
    for c in r.index: # get cols where correct response is defined
        if c in non_score_cols:
            continue
        
        # print(c, pts, type(r[c]))
        if c == 'start year':
            pts += sum(out[c].astype(float) <= r[c])/len(out)
        elif c == 'end year':
            pts += sum(r[c] <= out[c].astype(float))/len(out)
        elif r.isna()[c] or (type(r[c])==str and 'etc' in r[c]):
            # scoring a string with NaN/etc, count all as correct
            pts += 1 # len(out)
        else:
            if type(r[c])=='str' and ',' in r[c]:
                # scoring a string with limited set correct answers
                vals = set(r[c].split(', '))
            else:
                # scoring a string with one correct answer
                vals = set([r[c]])
            pts += sum(out[c].isin(vals))/len(out)
        
        # print(c, vals)
        # print(sum(out[c].isin(vals)))
# pts = pts/len(out)
print(valids, pts)

0 0


In [10]:
queries

,annotator,src,query,variable,frequency,start year,end year,MIP,experiment
0,Sonia,Us,What are some datasets relevant to extreme wea...,"pr, ua, etc",NaN,NaN,NaN,NaN,NaN
1,Sonia,Us,"Plot the 10, 20, 50, and 100 yr return period ...",pr,day,NaN,NaN,historical,NaN
2,Sonia,Us,Show me all the times where Ames Iowa has more...,tas,day,1980.0,2014.0,CMIP,historical
3,Sonia,Us,Show me the average annual number of days per ...,tasmax,day,2050.0,2070.0,ScenarioMIP,ssp<NNN>
4,Sonia,Us,You just showed me average annual number of da...,tasmax,day,2050.0,2070.0,ScenarioMIP,ssp<NNN>
5,Sonia,Us,Get the average annual number of days rainfall...,pr,day,1980.0,2014.0,CMIP,amip
6,Sonia,Us,"Show me in the future, all the suitable places...","clt, pr, etc",NaN,2015.0,NaN,ScenarioMIP,ssp<NNN>
7,Sonia,Us,Show me a future simulation where there is a d...,"pr, tas, etc",mon,2015.0,NaN,ScenarioMIP,ssp<NNN>
8,Sonia,Us,Tell me the average SPEI under SSP585 scenario...,"tas, pr",mon,2080.0,2100.0,ScenarioMIP,ssp585


In [11]:
len(dfmsub)

296962

In [14]:
conversation = {'input': "I'm interested in precipitation flux at a resolution of a month",
                'chat_history': []}
standalone = chain_standalone.invoke(conversation)
print(standalone)
pdagent.invoke('air temperature monthly resolution??')

precipitation flux resolution month


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['long_name'].str.contains('air temperature', case=False) & df['frequency'].str.contains('mon', case=False)]"}`


                MIP        model experiment variable  start year  end year  \
16             CMIP  GISS-E2-1-G    1pctCO2     tatp        1850      1900   
17             CMIP  GISS-E2-1-G    1pctCO2     tatp        1901      1950   
18             CMIP  GISS-E2-1-G    1pctCO2     tatp        1951      2000   
19             CMIP  GISS-E2-1-G    1pctCO2     tatp        2001      2019   
20             CMIP  GISS-E2-1-G    1pctCO2     ttop        1850      1900   
...             ...          ...        ...      ...         ...       ...   
296654  ScenarioMIP  GISS-E2-1-G     ssp585   tasmin        2051      2100   
296677  ScenarioMIP  GISS-E2-1-G     ssp585       ta        2015      2050   
296678  ScenarioMIP  GISS-E2-1-G     ssp585       ta     